# Modulación OFDM en Tiempo Discreto

## 1. Introducción

**Orthogonal Frequency Division Multiplexing (OFDM)** es una técnica de modulación digital que divide la información en múltiples subportadoras ortogonales, lo que permite transmitir datos a altas tasas de manera robusta ante interferencias y desvanecimientos selectivos en frecuencia. La utilización de la transformada inversa de Fourier (IFFT) permite generar la señal OFDM en el dominio del tiempo a partir de los símbolos modulados (por ejemplo, QPSK, QAM), facilitando su implementación en sistemas digitales.

## 2. Transmisión OFDM en Tiempo Discreto

El sistema OFDM en tiempo discreto se basa en dividir el flujo de datos en multiples subportadoras ortogonales, transmitiendo simultaneamente cada flujo de datos a traves de una subportadora diferente.
Para una mayor comprension de el proceso de transmision, decidimos separarlo en las siguientes etapas:

### Etapa N° 1: Conversion de Datos a Simbolos

Los datos de entrada (una cadena de numeros binarios) se agrupan en bloques de longitud fija y dependiendo de la técnica de modulación utilizada (QAM o PSK), cada grupo de bits se convierte en un símbolo complejo.

#### Modulacion QAM: 

Esta modulacion nos permite enviar más información a través de una sola señal, ya que se usan diferentes combinaciones de amplitudes y fases. Cuantos más niveles de amplitud y fase se utilicen, más datos se pueden transmitir en el mismo espacio de tiempo, pero también se hace más difícil para el receptor descifrar la señal si hay ruido o interferencia.

La ecuación de la función QAM es:

$$s(t) = I(t)\cdot\cos(2\pi f_ct)+Q(t)\cdot\sin(2\pi f_ct)$$

Donde:

- $I(t)$ es la componente en fase *(in-phase)* respecto a la portadora.
- $Q(t)$ es la componente en cuadratura *(quadrature)* respecto a la portadora (desfasada 90°).
- $f_c$ es la frecuencia de la portadora.

Para una primera instancia utilizaremos una modulacion 4-QAM (4 símbolos) esta tiene 2 bits por símbolo.

**Ejemplo:**

Si los bits de entrada son `01`, el simbolo complejo resultante sería $X_k = 1 + j$ utilizando un mapeo estandar (de izquierda a derecha de arriba para abajo).


### Estapa N°2: Mapeo de Simbolos a Subportadoras

Esta etapa consiste en distribuir cada símbolo complejo generado en la etapa anterior en diferentes subportadoras ortogonales. El objetivo es aprovechar el uso de multiples portadoras para transmitir la informacion de manera eficiente.

- **Asignacion de subportadoras:** Los simbolos se agrupan en bloques de longitud $N$, donde $N$ representa el numero de subportadoras.
- **Distribucion en frecuencia:** Cada simbolo se asocia a una subportadora de frecuencia diferente. Esto significa que si tenemos un ancho de banda total de $B$, el espaciado entre subportadoras es $\frac{B}{N}.$

### Etapa N°3: Modulacion OFDM mediante IFFT.

Se aplica la Transformada Inversa de Fourier Discreta al bloque de Simbolos, para obtener la señal en dominio temporal.

La señal transmitida en tiempo discreto se expresa como:

$$s(n)=\sum_{k=0}^{N-1}{X_k\cdot e^{\frac{j2\pi kn}{N}}}$$

Donde:

- $X_k$ es el simbolo modulado.
- $N$ es el numero total de portadoras.

### Etapa N°4: Adición del Prefijo Cíclico (CP)

Para evitar Interferencia Intersimbolo (ISI), se añade un prefijo ciclico al comienzo de cada simbolo OFDM. El prefijo cíclico es una copia de la última parte del símbolo OFDM.

### Ejemplo Conceptual:

Suponiendo el siguiente flujo de datos: `01001011`.

- **Etapa 1:** Los simbolos resultantes del flujo de datos son...

    - $X_0(01) = 1+j$
    - $X_1(00) = -1+j$
    - $X_2(10) = -1-j$
    - $X_3(11) = 1-j$

- **Etapa 2:** Suponiendo $N=4$ subportadoras...

    - $X_0 \rightarrow N_0$
    - $X_1 \rightarrow N_1$
    - $X_2 \rightarrow N_2$
    - $X_3 \rightarrow N_3$

- **Etapa 3:** La señal OFDM resultante será...

$$s(n)=(1+j)e^{j0}+(-1+j)e^{j\frac{2\pi n}{4}}+(-1-j)e^{j\frac{4\pi n}{4}}(1-j)e^{j\frac{6\pi n}{4}}$$

- **Etapa 4:** Suponiendo un prefijo ciclico de longitud 2 y 4 muestras...

    -  $[s(0),s(1),s(2),s(3)]\rightarrow[s(2),s(3),s(0),s(1),s(2),s(3)]$